# Splitting datasets into train, tune, and test sets
This notebook shows how to split datasets into train, tune, and test sets. The splits can be saved to a folder for reuse and reproducibility (recommended).

You can generate three types of splits. 
- A "super test" or withholding split. It's a simple random sample of variants meant to be completely held out until the final model training and evaluation.
- Classic train-tune-test splits based on percentages of the total dataset. 
- Reduced train size splits for testing model sensitivity to smaller training set sizes.

You can also write your own function to generate a split based on whatever criteria you like. As long as there's a "train" set (and a "tune" set if you're using early stopping), it should work with the rest of the codebase.

In [1]:
# reload modules before executing code in order to make development and debugging easier
%load_ext autoreload
%autoreload 2

In [2]:
# this jupyter notebook is running inside of the "notebooks" directory
# for relative paths to work properly, we need to set the current working directory to the root of the project
# for imports to work properly, we need to add the code folder to the system path
import os
from os.path import abspath, join, isdir
import sys
if not isdir("notebooks"):
    # if there's a "notebooks" directory in the cwd, we've already set the cwd so no need to do it again
    os.chdir("..")
module_path = abspath("code")
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import numpy as np
import constants
import split_dataset as sd
import utils

# set logging level to info
import logging
logging.basicConfig()
logger = logging.getLogger("nn4dms")
logger.setLevel(logging.INFO)

# Generating a "super test" set

I recommend using a completely held-out supertest set. Don't use this set for development of the algorithm and don't look at evaluation results on this set until the very end, when you are ready to publish. Here we will create a supertest set for avgfp and save it to the avgfp splits directory in data/avgfp/splits.

In [4]:
# load the full dataset (really, we only need the # of variants in the dataset, but this is easier)
ds = utils.load_dataset(ds_name="avgfp")

# specifying an out_dir will save 
supertest_idxs = sd.supertest(ds, size=.1, rseed=12, out_dir="data/avgfp/splits", overwrite=True)
supertest_idxs

INFO:nn4dms.split_dataset:saving supertest split to file data/avgfp/splits/supertest_wf333b1bd195c_s0.1_r12.txt


array([48689, 22582, 38827, ..., 34538, 27872, 39568])

# Generating a classic train-tune-test split
You must specify the size of each set as a fraction of the total number of examples. You can specify a fraction of 0 if you do not want to have a particular set. Train + tune + test must sum to 1. 

In [5]:
split = sd.train_tune_test(ds, train_size=.6, tune_size=.2, test_size=.2, rseed=12, out_dir="data/avgfp/splits", overwrite=True)

INFO:nn4dms.split_dataset:saving train-tune-test split to directory data/avgfp/splits/tr0.6_tu0.2_te0.2_wF_r12


In [6]:
split

{'tune': array([48689, 22582, 38827, ...,  3939, 32187, 29872]),
 'test': array([11269, 32373,  5628, ..., 29229,  3959, 25899]),
 'train': array([29742,  8044, 31001, ..., 51795, 49237, 30134])}

Note that these sets include the variants from the supertest set we created above. In order to generate a train-tune-test split without those variants, you must specify the either the supertest indices as an array or the filename if you saved them to the disk. 

In [7]:
supertest_fn = "data/avgfp/splits/supertest_wf333b1bd195c_s0.1_r12.txt"
split = sd.train_tune_test(ds, train_size=.6, tune_size=.2, test_size=.2, 
                           withhold=supertest_fn, rseed=12, out_dir="data/avgfp/splits", overwrite=True)
split

INFO:nn4dms.split_dataset:saving train-tune-test split to directory data/avgfp/splits/tr0.6_tu0.2_te0.2_wf333b1bd195c_r12


{'stest': array([48689, 22582, 38827, ..., 34538, 27872, 39568]),
 'tune': array([ 4223, 49475, 33378, ...,  4728, 49730, 51157]),
 'test': array([46162, 51625,  1508, ..., 52595, 26108, 24195]),
 'train': array([50942, 47964, 52040, ..., 19323, 21347,   656])}

The supertest set is added to the split as a "withheld" set, and the train, tune, and test sets do not contain any variants from it. The save directory contains an alphanumeric string "f333b1bd195c" representing a hash of the withheld indices. This is added to handle the edge case where you have multiple supertest sets and want to make multiple splits based on them.

# Generating a reduced train size split
Similar to generating a classic train-tune-test split, this function takes the size of the tune and test set as a fraction of the total data. The train set size is specified as a proportion of the data remaining after the tune and test sets have been selected. So, if a you specify a 20% tune and a 20% test size, that leaves a pool of 60% for selecting the test set. If your test set proportion is 50%, then the the function will randomly select half the variants from the 60% pool. You must also specify how many train samples of the given size you want. Using multiple samples is important because you may get a really good or really bad training set by chance, especially if you are using a small training size. 

In [8]:
splits = sd.reduced_train_size(ds, tune_size=.2, test_size=.2, train_prop=.5, num_train_reps=5,
                               withhold=supertest_fn, rseed=12, out_dir="data/avgfp/splits", overwrite=True)
splits

INFO:nn4dms.split_dataset:saving reduced split to directory data/avgfp/splits/reduced_tr0.5_tu0.2_te0.2_wf333b1bd195c_s5_r12


[{'stest': array([48689, 22582, 38827, ..., 34538, 27872, 39568]),
  'tune': array([ 4223, 49475, 33378, ...,  4728, 49730, 51157]),
  'test': array([46162, 51625,  1508, ..., 52595, 26108, 24195]),
  'train': array([23819, 27807, 10803, ...,  6478, 51121, 17221])},
 {'stest': array([48689, 22582, 38827, ..., 34538, 27872, 39568]),
  'tune': array([ 4223, 49475, 33378, ...,  4728, 49730, 51157]),
  'test': array([46162, 51625,  1508, ..., 52595, 26108, 24195]),
  'train': array([48548, 44186, 33607, ..., 20771, 52892,  2830])},
 {'stest': array([48689, 22582, 38827, ..., 34538, 27872, 39568]),
  'tune': array([ 4223, 49475, 33378, ...,  4728, 49730, 51157]),
  'test': array([46162, 51625,  1508, ..., 52595, 26108, 24195]),
  'train': array([ 2098, 51006, 28612, ...,   438, 17146, 53031])},
 {'stest': array([48689, 22582, 38827, ..., 34538, 27872, 39568]),
  'tune': array([ 4223, 49475, 33378, ...,  4728, 49730, 51157]),
  'test': array([46162, 51625,  1508, ..., 52595, 26108, 24195]),


Since multiple train replicates were specified, the result is a list of splits. Note that the tune and test sets are exactly the same as the previous section. This is because we used the same random seed as above and is the desired behavior.

# Loading a saved split

In [9]:
split_dir = "data/avgfp/splits/tr0.6_tu0.2_te0.2_wf333b1bd195c_r12"
split = sd.load_split_dir(split_dir)
split

{'train': array([50942, 47964, 52040, ..., 19323, 21347,   656]),
 'stest': array([48689, 22582, 38827, ..., 34538, 27872, 39568]),
 'test': array([46162, 51625,  1508, ..., 52595, 26108, 24195]),
 'tune': array([ 4223, 49475, 33378, ...,  4728, 49730, 51157])}

In [10]:
reduced_split_dir = "data/avgfp/splits/reduced_tr0.5_tu0.2_te0.2_wf333b1bd195c_s5_r12"
splits = sd.load_split_dir(reduced_split_dir)
splits

[{'train': array([18875, 51882, 30975, ...,  1059, 33408, 14718]),
  'stest': array([48689, 22582, 38827, ..., 34538, 27872, 39568]),
  'test': array([46162, 51625,  1508, ..., 52595, 26108, 24195]),
  'tune': array([ 4223, 49475, 33378, ...,  4728, 49730, 51157])},
 {'train': array([50373, 40181, 10151, ..., 42430, 25024, 52827]),
  'stest': array([48689, 22582, 38827, ..., 34538, 27872, 39568]),
  'test': array([46162, 51625,  1508, ..., 52595, 26108, 24195]),
  'tune': array([ 4223, 49475, 33378, ...,  4728, 49730, 51157])},
 {'train': array([ 2098, 51006, 28612, ...,   438, 17146, 53031]),
  'stest': array([48689, 22582, 38827, ..., 34538, 27872, 39568]),
  'test': array([46162, 51625,  1508, ..., 52595, 26108, 24195]),
  'tune': array([ 4223, 49475, 33378, ...,  4728, 49730, 51157])},
 {'train': array([23819, 27807, 10803, ...,  6478, 51121, 17221]),
  'stest': array([48689, 22582, 38827, ..., 34538, 27872, 39568]),
  'test': array([46162, 51625,  1508, ..., 52595, 26108, 24195]),